<a href="https://colab.research.google.com/github/poojashreeNS/Dimensionality_Reduction/blob/main/Dimensionality_Reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
# For plotting
import plotly.io as plt_io
import plotly.graph_objects as go
%matplotlib inline

In [ ]:
!pip install umap-learn[plot]
!pip install holoviews
!pip install -U ipykernel

In [27]:
#PCA
from sklearn.decomposition import PCA
#TSNE
from sklearn.manifold import TSNE
#UMAP
import umap
#LLE
from sklearn.manifold import LocallyLinearEmbedding as LLE
#Isomap
from sklearn.manifold import Isomap
#SVD
from sklearn.decomposition import TruncatedSVD

In [8]:
train = pd.read_csv('/content/sign_mnist_test.csv')
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,6,149,149,150,150,150,151,151,150,151,152,152,152,152,152,153,153,151,152,152,153,152,152,151,151,150,150,150,149,150,150,150,152,152,151,152,152,152,152,152,...,131,134,144,147,125,87,87,103,107,110,116,113,75,74,74,74,76,74,82,134,168,155,146,137,145,146,149,135,124,125,138,148,127,89,82,96,106,112,120,107
1,5,126,128,131,132,133,134,135,135,136,138,137,137,138,138,139,137,142,140,138,139,137,137,136,135,134,133,134,132,129,132,134,135,135,137,139,139,139,140,141,...,114,112,89,48,133,194,182,185,184,184,182,181,172,174,177,178,178,179,181,183,187,175,165,154,118,107,100,75,96,83,47,104,194,183,186,184,184,184,182,180
2,10,85,88,92,96,105,123,135,143,147,152,157,163,168,171,182,172,175,185,183,184,185,185,185,183,183,182,181,178,86,88,93,96,108,125,137,145,149,154,160,...,145,123,78,162,239,227,229,226,226,225,224,222,89,91,94,111,136,154,167,184,125,3,166,225,195,188,172,185,161,122,68,166,242,227,230,227,226,225,224,222
3,0,203,205,207,206,207,209,210,209,210,209,208,207,207,209,208,210,210,207,209,209,208,209,210,209,207,208,209,207,206,208,209,208,208,210,211,210,211,209,209,...,85,80,84,151,238,255,255,250,237,245,250,232,103,101,102,103,95,208,231,227,209,190,179,182,152,150,159,119,83,63,154,248,247,248,253,236,230,240,253,255
4,3,188,191,193,195,199,201,202,203,203,203,204,204,204,203,202,198,216,217,135,181,200,195,194,193,190,189,187,185,190,194,196,197,200,202,204,206,207,207,206,...,93,52,24,53,63,33,41,51,48,45,49,55,149,150,150,148,147,151,124,82,84,81,69,81,111,103,84,75,53,28,26,40,64,48,29,46,49,46,46,53


In [9]:
#picking only the first 10 labels
train = train[train['label'] < 10]
# Setting the label and the feature columns
y = train.loc[:,'label'].values
x = train.loc[:,'pixel1':].values

Implementing Plotting functions

In [10]:
def plot_2d(component1, component2):
    
    fig = go.Figure(data=go.Scatter(
        x = component1,
        y = component2,
        mode='markers',
        marker=dict(
            size=10,
            color=y, #set color equal to a variable
            colorscale='Rainbow', # one of plotly colorscales
            showscale=True,
            line_width=1
        )
    ))
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),width=2000,height=1200)                 
    fig.layout.template = 'plotly_dark'
    
    fig.show()

In [11]:
def plot_3d(component1,component2,component3):

  fig = go.Figure(data=[go.Scatter3d(
        x=component1,
        y=component2,
        z=component3,
        mode='markers',
        marker=dict(
            size=5,
            color=y,                # set color to an array/list of desired values
            colorscale='Rainbow',   # choose a colorscale
            opacity=1,
            line_width=1
        )
    )])
  # tight layout
  fig.update_layout(margin=dict(l=50,r=50,b=50,t=50),width=1800,height=1000)
  fig.layout.template = 'plotly_dark'
    
  fig.show()

Standardizing dataset

In [12]:
from sklearn.preprocessing import StandardScaler
## Standardizing the data
x = StandardScaler().fit_transform(x)

# **Implementing PCA**

In [ ]:
start = time.time()
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)
print('Duration: {} seconds'.format(time.time() - start))
principal = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3'])

Duration: 0.3520987033843994 seconds


PCA_2D

In [ ]:
plot_2d(principalComponents[:, 0],principalComponents[:, 1])

PCA_3D

In [ ]:
plot_3d(principalComponents[:, 0],principalComponents[:, 1],principalComponents[:, 2])

# **Implementing t-SNE**

In [ ]:
start = time.time()
pca_50 = PCA(n_components=50)
pca_result_50 = pca_50.fit_transform(x)
tsne = TSNE(random_state = 42, n_components=3,verbose=0, perplexity=40, n_iter=400).fit_transform(pca_result_50)
print('Duration: {} seconds'.format(time.time() - start))

Duration: 37.894346475601196 seconds


t-SNE_2D

In [ ]:
plot_2d(tsne[:, 0],tsne[:, 1])

t-SNE_3D

In [ ]:
plot_3d(tsne[:, 0],tsne[:, 1],tsne[:, 2])

# **Implementation of UMAP**

In [ ]:
start = time.time()
reducer = umap.UMAP(random_state=42,n_components=3)
embedding = reducer.fit_transform(x)
print('Duration: {} seconds'.format(time.time() - start))

UMAP_2D

In [ ]:
plot_2d(reducer.embedding_[:, 0],reducer.embedding_[:, 1])

UMAP_3D

In [ ]:
plot_3d(reducer.embedding_[:, 0],reducer.embedding_[:, 1],reducer.embedding_[:, 2])

# **Implementation of LLE**

In [23]:
def run_lle(num_neighbors, dims, mthd, data):
    # Specify LLE parameters
    embed_lle = LLE(n_neighbors=num_neighbors,
                    n_components=dims, 
                    reg=0.001,
                    eigen_solver='auto',
                    method=mthd, 
                    modified_tol=1e-12,
                    neighbors_algorithm='auto',
                    random_state=42,
                    n_jobs=-1
                   )
    # Fit and transofrm the data
    result = embed_lle.fit_transform(data)
    
    # Return results
    return result

LLE_2D

In [14]:
std_lle_res=run_lle(num_neighbors=30, dims=2, mthd='standard', data=x)
plot_2d(std_lle_res[:, 0],std_lle_res[:, 1] )

LLE_3D

In [18]:
std_lle_res=run_lle(num_neighbors=30, dims=3, mthd='modified', data=x)
plot_3d(std_lle_res[:, 0], std_lle_res[:, 1], std_lle_res[:, 2])

# **Implementation of Isomap**

In [22]:
def run_isomap(num_neighbors, dims, data):
    # Specify Isomap parameters
    embed_isomap = Isomap(n_neighbors=num_neighbors, n_components=dims, n_jobs=-1)
    
    # Fit and transofrm the data
    result = embed_isomap.fit_transform(data)
    
    # Return results
    return result    

Isomap_2D

In [24]:
isomap_res=run_isomap(num_neighbors=30, dims=2, data=x)
plot_2d(isomap_res[:, 0], isomap_res[:, 1])

[[ -2.57952027  45.55779049]
 [ 19.3330679    2.35683691]
 [-11.4473394  -23.24483809]
 ...
 [ 10.76698079 -17.27879299]
 [-18.93096011 -51.2927459 ]
 [ 28.86483432  -5.60117552]]


Isomap_3D

In [25]:
isomap_res=run_isomap(num_neighbors=30, dims=3, data=x)
plot_3d(isomap_res[:,0], isomap_res[:, 1], isomap_res[:, 2])

[[ -2.57952027  45.55779049 -11.10928201]
 [ 19.3330679    2.35683691   2.68400811]
 [-11.4473394  -23.24483809   4.37008567]
 ...
 [ 10.76698079 -17.27879299 -43.85010264]
 [-18.93096011 -51.2927459    3.15320809]
 [ 28.86483432  -5.60117552 -43.55734031]]


# **Implementation of SVD**

In [31]:
svd = TruncatedSVD(n_components=2)
X_reduced = svd.fit_transform(x)
plot_2d(X_reduced[:, 0], X_reduced[:, 1])

In [33]:
svd = TruncatedSVD(n_components=3)
X_reduced = svd.fit_transform(x)
plot_3d(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2])